In [13]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import pandas as pd
import re  # 숫자추출을 위한 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyperclip # 자동 로그인을 위한 변수 복사 라이브러리
from time import sleep


In [15]:
def create_excel(time, rank, server) : 
# 엑셀 만들기
    wb = openpyxl.Workbook()

    # 워크시트 만들기
    ws = wb.active
    ws.title = f"최근 {time}일 {server}서버의 {rank}랭크"

    # 데이터 추가하기
    ws.append(['직업', '덱유형', '승률', '점유율', '게임수'])

    # 엑셀 저장하기
    wb.save('./hs_meta.xlsx')



In [24]:
def driver_in(): 
# ChromeOptions 설정
  chrome_options = Options()
  chrome_options.add_experimental_option("detach", True)
  chrome_options.add_argument("--incognito")
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  # ChromeDriverManager를 사용하여 Chrome 드라이버를 초기화합니다.
  driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=chrome_options)

  # url 열기
  url = "https://hsreplay.net/account/login"
  driver.get(url)
  
  # 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
  try:
      element_present = EC.presence_of_element_located((By.CLASS_NAME, "promo-button"))
      WebDriverWait(driver, 10).until(element_present)
  except Exception as e:
      print(f"엘리먼트 대기 중 예외 발생1: {e}")
      driver.quit()

  btn = driver.find_element(By.CLASS_NAME,"promo-button")
  btn.click()

  # 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
  try:
      element_present = EC.presence_of_element_located((By.CLASS_NAME, "input-block"))
      WebDriverWait(driver, 10).until(element_present)
  except Exception as e:
      print(f"엘리먼트 대기 중 예외 발생2: {e}")
      driver.quit()

  id = driver.find_elements(By.CLASS_NAME, "input-block")[0]
  pyperclip.copy("") # ID 복사
  id.send_keys(Keys.CONTROL, 'v') # ID 붙여넣기
  sleep(1)
  pw = driver.find_elements(By.CLASS_NAME, "input-block")[1]
  pyperclip.copy("") # 패스워드 복사
  pw.send_keys(Keys.CONTROL, 'v') # 패스워드 붙여넣기
  sleep(30)
  log_btn = driver.find_element(By.CLASS_NAME, "submit-button")
  log_btn.click()

  return driver, chrome_options

In [18]:
def scrape_hs_meta_data():
  # 입력값 받기
  time = int(input("검색할 기간을 입력하세요.\n[최근7일 : 7 // 최근 3일 : 3 // 최근 1일 : 1 // 최신 패치 : 0]"))
  rank = input("검색할 랭크를 입력하세요.\n[전구간 : A // 브론즈 : B // 실버 : S // 골드 : G // 플래티넘 : P // 다이아 : D // 전설 : L // 상위1000 : 1000]")
  server = input("검색할 서버를 입력하세요.\n[전서버 : A // 아시아 : AP // 유럽 : EU // 북미 : US]")

  # 검색 범위 딕셔너리 정의
  time_dict={7 : "LAST_7_DAY", 3 :  "LAST_3_DAY", 1 : "LAST_1_DAY", 0 : ""}
  rank_dict={"A" : "", "B" :  "BRONZE", "S" : "SILVER", "G" :  "GOLD", "P" :  "PLATINUM", "D" :  "DIAMOND", "L" :  "LEGEND", "1000" :  "TOP_1000_LEGEND"}
  server_dict={"A" : "", "AP" :  "REGION_KR", "EU" : "REGION_EU", "US" : "REGION_US"}

  # 엑셀 생성 함수
  create_excel(time, rank_dict[rank], server_dict[server])


  # 웹브라우저 진입 함수
  driver, chrome_opteions = driver_in()
  sleep(3) # 로그인 진입을 위한 대기시간 3초


  # URL 열기
  if rank_dict[rank]=="":
    url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time_dict[time]}&region={server_dict[server]}"
  else:
    url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time_dict[time]}&rankRange={rank_dict[rank]}&region={server_dict[server]}"
  driver.get(url)

  # 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
  try:
    element_present = EC.presence_of_element_located((By.CLASS_NAME, 'class-box-container'))
    WebDriverWait(driver, 10).until(element_present)
  except Exception as e:
    print(f"엘리먼트 대기 중 예외 발생3: {e}")
    driver.quit()



  # BeautifulSoup을 사용하여 HTML 파싱
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')

  # 직업 클래스
  elements = driver.find_elements(By.CSS_SELECTOR, '.class-box')

  # 엑셀 열기
  wb = openpyxl.load_workbook('./hs_meta.xlsx')
  ws = wb.active

  # 데이터를 저장할 리스트 생성
  data = []

  # 메인 루프
  for element in elements:
      # 직업 가져오기
      hero = element.find_element(By.CSS_SELECTOR, '.box-title').text
      print(f"직업 {hero}")
      
      # 해당 직업에 대한 덱 유형과 정보 클래스
      deck_elements = element.find_elements(By.CSS_SELECTOR, '.table-row-header')
      stats_elements = element.find_elements(By.CSS_SELECTOR, '.table-cell')


      for deck_element in deck_elements:
        # 덱 유형 가져오기
        type_name = deck_element.find_element(By.CSS_SELECTOR, '.tooltip-wrapper').text
        print(f"덱유형 {type_name}")

        # 승률, 점유율, 게임 수 가져오기
        win_rate = stats_elements[0].text
        popularity = stats_elements[1].text
        games = stats_elements[2].text
        print(f"승률: {win_rate}, 점유율: {popularity}, 게임 수: {games}")
      
        # 데이터를 리스트에 추가
        data.append([hero, type_name, win_rate, popularity, games])

        # stats_elements에서 현재 처리한 스탯 정보를 제거
        stats_elements = stats_elements[3:]

  # Chrome 드라이버 종료
  driver.quit()
  # .xlsx 형식으로 엑셀에 추가
  for row in data:
      ws.append(row)

  # .xlsx 형식으로 엑셀 저장
  wb.save('./hs_meta.xlsx')

  # 데이터 리스트 반환
  return data
      

In [25]:
# 함수 호출하여 데이터 수집
scraped_data = scrape_hs_meta_data()

# 데이터를 포함한 리스트에서 DataFrame을 생성
df = pd.DataFrame(scraped_data, columns=['Hero', 'Deck', 'Winrate', 'Popular', 'Games'])
print("데이터 프레임",df)
# DataFrame을 UTF-8-sig 인코딩으로 CSV 파일로 저장
df.to_csv('./hs_meta.csv', encoding='utf-8-sig', index=False)

직업 죽음의 기사
덱유형 Plague Death Knight
승률: 57.1%, 점유율: 6.7%, 게임 수: 13,000
덱유형 부정 죽음의 기사
승률: 38.7%, 점유율: 4.1%, 게임 수: 8,400
덱유형 혈기 죽음의 기사
승률: 43.3%, 점유율: 2.5%, 게임 수: 5,100
덱유형 냉기 죽음의 기사
승률: 48.3%, 점유율: 1.0%, 게임 수: 1,900
덱유형 Rainbow Death Knight
승률: 48.5%, 점유율: 0.5%, 게임 수: 1,000
덱유형 Highlander Blood Death Knight
승률: 56.2%, 점유율: 0.4%, 게임 수: 880
덱유형 기타
승률: 45.6%, 점유율: 1.4%, 게임 수: 2,900
직업 악마사냥꾼
덱유형 어그로 악마사냥꾼
승률: 57.3%, 점유율: 2.6%, 게임 수: 5,400
덱유형 빅 악마사냥꾼
승률: 34.6%, 점유율: 0.4%, 게임 수: 760
덱유형 하이랜더 악마사냥꾼
승률: 49.4%, 점유율: 0.2%, 게임 수: 430
덱유형 유물 악마사냥꾼
승률: 30.9%, 점유율: 0.2%, 게임 수: 350
덱유형 기타
승률: 23.8%, 점유율: 0.5%, 게임 수: 990
직업 드루이드
덱유형 용 드루이드
승률: 61.5%, 점유율: 5.5%, 게임 수: 11,000
덱유형 하이랜더 드루이드
승률: 58.7%, 점유율: 2.9%, 게임 수: 5,900
덱유형 나무정령 드루이드
승률: 61.0%, 점유율: 1.1%, 게임 수: 2,200
덱유형 기타
승률: 37.1%, 점유율: 1.2%, 게임 수: 2,300
직업 사냥꾼
덱유형 사냥개 사냥꾼
승률: 60.8%, 점유율: 3.2%, 게임 수: 6,500
덱유형 비전 사냥꾼
승률: 58.1%, 점유율: 1.2%, 게임 수: 2,500
덱유형 하이랜더 사냥꾼
승률: 58.8%, 점유율: 0.9%, 게임 수: 1,700
덱유형 기타
승률: 44.9%, 점유율: 1.6%, 게임 수: 3,200
직업 마법사
덱유형 R